# 리디북스 - CSS Selector 이용

In [66]:
import requests
from bs4 import BeautifulSoup

req=requests.get('https://ridibooks.com/books/875103701')
source=req.text
soup=BeautifulSoup(source, 'html.parser')

# 제목
title_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div.info_title_wrap > h3")
title=title_list[0].text
title=title[5:] #'[연재] '가 제목 앞에 붙음

# 저자
author_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_writer > span > a")
author=author_list[0].text

# 출판사
publisher_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.file_info.publisher_info > a")
publisher=publisher_list[0].text

# 총 연재화수 - 정수
book_count_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_info_series_complete_wrap > span.metadata_item.book_count")
book_count=book_count_list[0].text
book_count=book_count[2:len(book_count)-1]
book_count=int(book_count)

# 완결 여부
complete_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_info_series_complete_wrap > span.metadata_item.not_complete")
if complete_list[0].text=='미완결':
    complete='미완결'
else:
    complete='완결'

# 출간일
published_date_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.Header_Metadata_Block > ul > li.Header_Metadata_Item.book_info.published_date_info > ul > li")
published_date=published_date_list[0].text
published_date=published_date[2:13]
    
# 최근 업데이트일
recent_update_list=soup.select("#SeriesListWrap > ul.module_compact_book_list.white_theme.js_compact_book_list > li > div > div > div.table_cell.book_info > div > ul > li.info_reg_date")
update=recent_update_list[book_count-1].text
update=update[6:16]

# 표지 사진
img=soup.find("img")
img_src=img.get("src")

# 카테고리
category_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > p > a")
category=category_list[1].text

# 별점 점수
star_rate_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p > span > span.StarRate_Score")
star_rate=star_rate_list[0].text

# 별점 참여자
star_rate_count_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p > span > span.StarRate_ParticipantCount")
star_rate_count=star_rate_count_list[0].text

# 관심도
preference_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_thumbnail_wrap > div.header_preference > button > span > span.button_text.js_preference_count")
preference=preference_list[0].text

# 책소개
introduction_list=soup.select("#introduce_book > p")
introduction=introduction_list[2].text
introduction= introduction.replace("<[연재] "+title+"> ","")
introduction= introduction.replace("\r","")
introduction= introduction.replace("책 소개","")
introduction= introduction.replace("펼쳐보기 ","")

############################## 결과 출력 #############################

print("제목: " + title)
print("저자: " + author)
print("출판사: " + publisher)
print("표지 url: " + img_src)
print("총 연재화수: ", book_count)
print("완결 여부: " + complete)
print("출간일: " + published_date)
print("최근 업데이트일: " + update)
print("장르: " + category)
print("별점: " + star_rate)
print("별점 참여자: " + star_rate_count)
print("관심도: " + preference)
print("소개: " + introduction)

제목: 백작가의 망나니가 되었다
저자: 유려한
출판사: 도서출판 청어람
표지 url: //img.ridicdn.net/cover/875125926/xxlarge
총 연재화수:  574
완결 여부: 미완결
출간일: 2018.10.02.
최근 업데이트일: 2020.05.08
장르: 퓨전 판타지
별점: 4.8점
별점 참여자: 3,929명
관심도: 0
소개: 눈을 떠보니 소설 속이었다.
그것도 망나니로 유명한 백작가 도련님 몸으로.

하지만,
그렇다고 망나니가 될 순 없잖아?


# 조아라 - Selenium 이용

In [221]:
import urllib.request
from selenium import webdriver
import bs4
import requests
from urllib import parse

url = "http://www.joara.com/premium_new/book_intro.html?book_code=1445186"
driver = webdriver.Chrome("C:/project/chromedriver.exe")
driver.get(url)

html=driver.page_source
soup=BeautifulSoup(html, 'lxml')

# 표지 사진
img_class = soup.find(name="div", attrs={"class":"img_s"})
img_tag = img_class.find_all('img')
url_list = []
for a in img_tag:
    url_list.append(a.get('src'))
img_url=url_list[0]

info_class = soup.find(name="div", attrs={"class":"txt_c_sty01"})

# 제목
title_class= info_class.find('a')
title=title_class.text

# 카테고리
category= info_class.strong.text.split(' ')[0].replace("[","")
category= category.replace("]","")

#저자
author_class= info_class.find('span')
author=author_class.text
author= author.replace(" ","")
author= author.replace("\n","")
author= author.replace("/","")

# 총 연재화수
book_count_class = info_class.find(name="div", attrs={"class":"work_view"})
book_count_class= book_count_class.find(name="span", attrs={"class":"select"})
book_count=book_count_class.text.replace("\n","")

# 최근 업데이트일
last_date_class = info_class.find(name="div", attrs={"class":"info_c"})
last_date_class= last_date_class.find(name="span", attrs={"class":"date"})
last_date=last_date_class.text
update=last_date[:10]+'.'

# 완결 여부
complete_class = info_class.find(name="div", attrs={"class":"info_c"})
complete_class= complete_class.find(name="strong")
if complete_class.text=='마지막 연재일':
    complete='미완결'
else:
    complete='완결'

# 조회수, 추천, 선호작
info2_class = info_class.find(name="p", attrs={"class":"info2"})
info2_class = info2_class.find_all(name="span", attrs={"class":"date"})
views = info2_class[0].text.replace(",","") # 조회수
recommend = info2_class[1].text # 추천
like = info2_class[2].text # 선호작

# 책소개
introduction_class = soup.find(name="div", attrs={"class":"t_cont_v"})
introduction=introduction_class.text.replace("\t","")
introduction=introduction.replace("\n","")

############################## 결과 출력 ############################

print("제목: " + title)
print("장르: " + category)
print("저자: " + author)
print("표지 url: " + img_url)
print("총 연재화수: ", book_count)
print("최근 업데이트일: " + update)
print("완결 여부: " + complete)
print("조회수: " + views)
print("추천수: " + recommend)
print("관심도: " + like)
print("소개: " + introduction)

# print("출판사: " + publisher) : 안내 없음
# print("별점: " + star_rate)
# print("별점 참여자: " + star_rate_count)
# print("출간일: " + published_date)

driver.close()

제목: 튜토리얼 오류로 최강
장르: 판타지
저자: [북극펭귄]
표지 url: http://cf.joara.com/literature_file/20200428_1848521675.jpg_thumb.png
총 연재화수:  244편
최근 업데이트일: 2020.04.29.
완결 여부: 완결
조회수: 132908
추천수: 506
관심도: 724
소개: 아팠다. 이미 수십 번은 차인 곳이었는데 아프기는 매번 지독하게 아팠다.“네가 내 것이 되지 못한다면…….”머리에 단어가, 문장이 울려퍼진다. 마치 신의 목소리처럼.“그냥 자살해 줘.”...눈을 떴다.[그럼 지금부터 카운트에 들어갑니다. 한 시간 안에 자신이 도전할 난이도를 선택한 후 문을 열어 주십시오.]‘죽음의 시련’난이도 10 시련의 또 하나의 이름.생존률이 0%의 시련.그곳으로 들어간다.


## 조아라 - CSS Selector 이용

In [331]:
import requests
from bs4 import BeautifulSoup

req=requests.get('http://www.joara.com/premium_new/book_intro.html?book_code=1360670')
source=req.text
soup=BeautifulSoup(source, 'html.parser')

# 제목
title_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong > em > a")
title=title_list[0].text

# 저자
author_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong > span > span")
author=author_list[0].text

# 총 연재화수 - 정수
book_count_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.work_view > span.select > span")
book_count=book_count_list[0].text
book_count=book_count.replace("편","")
book_count=int(book_count)

# 완결 여부
complete_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info > strong")
if complete_list[0].text=='마지막 연재일':
    complete='미완결'
else:
    complete='완결'

# 출간일
published_date_list=soup.find("table")
published_date_list=published_date_list.find_all("tr")
published_date_list=published_date_list[book_count]
published_date_list=published_date_list.find_all("a")
published_date=published_date_list[2].text
    
# 최근 업데이트일
recent_update_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info > span")
update=recent_update_list[0].text
update=update[:10]+'.'

# 표지 사진 url
img=soup.find("div")
img=soup.find(class_="img_s")
img=img.find("img")
img_src=img.get("src")

# 장르
category_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong")
category=category_list[0].text
category= category.split(' ')[0].replace("[","")
category= category.replace("]","")

# 조회수 - 정수
count_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info2 > span")
views=count_list[0].text
views=views.replace(",","")
views=int(views)

# 추천수 - 정수
recommend=count_list[2].text
recommend=recommend.replace(",","")
recommend=int(recommend)

# 관심도 - 정수
preference=count_list[4].text
preference=preference.replace(",","")
preference=int(preference)

# 책소개
introduction_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.t_cont_v")
introduction=introduction_list[0].text
introduction= introduction.replace("\t","")
introduction= introduction.replace("\r","")
#introduction= introduction.replace("\n","")

############################## 결과 출력 #############################

print("제목: " + title)
print("장르: " + category)
print("저자: " + author)
print("표지 url: " + img_src)
print("총 연재화수: ", book_count)
print("최근 업데이트일: " + update)
print("출간일: " + published_date)
print("완결 여부: " + complete)
print("조회수: ", views)
print("추천수: ", recommend)
print("관심도: ", like)
print("소개: " + introduction)

# print("출판사: " + publisher) : 안내 없음
# print("별점: " + star_rate)
# print("별점 참여자: " + star_rate_count)

제목: 무한서고의 계약자!
장르: 판타지
저자: 준솔
표지 url: http://cf.joara.com/literature_file/20190418_121420.jpg_thumb.png
총 연재화수:  220
최근 업데이트일: 2019.06.04.
출간일: 19/04/03
완결 여부: 완결
조회수:  613218
추천수:  3965
관심도:  724
소개: 
스킬북을 포함한 모든 서적들이 기록되어 있는 무한서고. 
나는 그런 무한서고를 열람할 수 있는 권능을 얻게 되는데···.


## 검색 링크 - 리디

In [1]:
import requests
from bs4 import BeautifulSoup

def get_info(url, book_id):
    print(url)
    print("리디북스 ID no. " + book_id)
    print("\n-------------------------\n")
    req=requests.get(ridi_url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')

    # 제목
    title_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div.info_title_wrap > h3")
    title=title_list[0].text
    title=title[5:] #'[연재] '가 제목 앞에 붙음

    # 저자
    author_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_writer > span > a")
    author=author_list[0].text

    # 출판사
    publisher_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.file_info.publisher_info > a")
    publisher=publisher_list[0].text

    # 총 연재화수 - 정수
    book_count_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_info_series_complete_wrap > span.metadata_item.book_count")
    book_count=book_count_list[0].text
    book_count=book_count[2:len(book_count)-1]
    book_count=int(book_count)

    # 완결 여부
    complete_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_info_series_complete_wrap > span.metadata_item.not_complete")
    if complete_list[0].text=='미완결':
        complete='미완결'
    else:
        complete='완결'

    # 출간일
    published_date_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.Header_Metadata_Block > ul > li.Header_Metadata_Item.book_info.published_date_info > ul > li")
    published_date=published_date_list[0].text
    published_date=published_date[2:13]
    
    # 최근 업데이트일
    recent_update_list=soup.select("#SeriesListWrap > ul.module_compact_book_list.white_theme.js_compact_book_list > li > div > div > div.table_cell.book_info > div > ul > li.info_reg_date")
    update=recent_update_list[book_count-1].text
    update=update[6:16]

    # 표지 사진
    img=soup.find("img")
    img_src=img.get("src")

    # 카테고리
    category_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > p > a")
    category=category_list[1].text

    # 별점 점수
    star_rate_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p > span > span.StarRate_Score")
    star_rate=star_rate_list[0].text

    # 별점 참여자
    star_rate_count_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p > span > span.StarRate_ParticipantCount")
    star_rate_count=star_rate_count_list[0].text

    # 관심도
    preference_list=soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_thumbnail_wrap > div.header_preference > button > span > span.button_text.js_preference_count")
    preference=preference_list[0].text

    # 책소개
    introduction_list=soup.select("#introduce_book > p")
    for j in introduction_list:
        introduction=j.text
        if(introduction.find("<[연재] "+title+">")>=0):
            introduction= introduction.replace("<[연재] "+title+"> ","")
            introduction= introduction.replace("\r","")
            introduction= introduction.replace("책 소개","")
            introduction= introduction.replace("펼쳐보기 ","")
            break;

    ############################## 결과 출력 #############################

    print("제목: " + title)
    print("저자: " + author)
    print("출판사: " + publisher)
    print("표지 url: " + img_src)
    print("총 연재화수: ", book_count)
    print("완결 여부: " + complete)
    print("출간일: " + published_date)
    print("최근 업데이트일: " + update)
    print("장르: " + category)
    print("별점: " + star_rate)
    print("별점 참여자: " + star_rate_count)
    print("관심도: " + preference)
    print("소개: " + introduction)

#################################################################################

ridi_base="https://ridibooks.com/search/?q="
ridi_work=input("작품 검색 : ")
print("\n-------------------------\n")
ridi_search=ridi_base+ridi_work.replace(" ","+")

req=requests.get(ridi_search)
source=req.text
soup=BeautifulSoup(source, 'html.parser')

url_list=soup.select("#page_search_result > div.result_list_wrapper > article > div.book_macro_wrapper.js_book_macro_wrapper > div")
check=0
for i in url_list:
    if(i.text.find("연재")>0):
        ridi_bookID=i.find(class_="book_thumbnail_wrapper")
        ridi_bookID=ridi_bookID.get("data-book_id_for_tracking")
        ridi_url="https://ridibooks.com/books/"+ridi_bookID
        get_info(ridi_url,ridi_bookID)
        check=1
        #print(book_id)
        break;

if check==0:
    print("검색 결과가 없습니다.")
else:
    print("\n-------------------------\n\n검색 결과 출력 완료")

작품 검색 : 마운드의 마에스트로

-------------------------

검색 결과가 없습니다.


## 검색 링크 - 조아라

In [103]:
import requests
from bs4 import BeautifulSoup

def get_info(url, book_id):
    print(url)
    print("조아라 ID no. " + book_id)
    print("\n-------------------------\n")
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')

    # 제목
    title_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong > em > a")
    title=title_list[0].text

    # 저자
    author_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong > span > span")
    author=author_list[0].text

    # 총 연재화수 - 정수
    book_count_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.work_view > span.select > span")
    book_count=book_count_list[0].text
    book_count=book_count.replace("편","")
    book_count=int(book_count)

    # 완결 여부
    complete_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info > strong")
    if complete_list[0].text=='마지막 연재일':
        complete='미완결'
    else:
        complete='완결'

    # 출간일
    published_date_list=soup.find("table")
    published_date_list=published_date_list.find_all("tr")
    published_date_list=published_date_list[book_count]
    published_date_list=published_date_list.find_all("a")
    published_date=published_date_list[2].text
    
    # 최근 업데이트일
    recent_update_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info > span")
    update=recent_update_list[0].text
    update=update[:10]+'.'

    # 표지 사진 url
    img=soup.find("div")
    img=soup.find(class_="img_s")
    img=img.find("img")
    img_src=img.get("src")

    # 장르
    category_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong")
    category=category_list[0].text
    category= category.split(' ')[0].replace("[","")
    category= category.replace("]","")

    # 조회수 - 정수
    count_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info2 > span")
    views=count_list[0].text
    views=views.replace(",","")
    views=int(views)

    # 추천수 - 정수
    recommend=count_list[2].text
    recommend=recommend.replace(",","")
    recommend=int(recommend)

    # 관심도 - 정수
    preference=count_list[4].text
    preference=preference.replace(",","")
    preference=int(preference)

    # 책소개
    introduction_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.t_cont_v")
    introduction=introduction_list[0].text
    introduction= introduction.replace("\t","")
    introduction= introduction.replace("\r","")
    #introduction= introduction.replace("\n","")

    ############################## 결과 출력 #############################

    print("제목: " + title)
    print("장르: " + category)
    print("저자: " + author)
    print("표지 url: " + img_src)
    print("총 연재화수: ", book_count)
    print("최근 업데이트일: " + update)
    print("출간일: " + published_date)
    print("완결 여부: " + complete)
    print("조회수: ", views)
    print("추천수: ", recommend)
    print("관심도: ", preference)
    print("소개: " + introduction)

    # print("출판사: " + publisher) : 안내 없음
    # print("별점: " + star_rate)
    # print("별점 참여자: " + star_rate_count)
    
##########################################################################

joara_base="http://www.joara.com/search/search.html?sl_search=book&sl_keyword="
joara_work=input("작품 검색 : ")
print("\n-------------------------\n")
joara_search=joara_base+joara_work.replace(" ","+")

req=requests.get(joara_search)
source=req.text
soup=BeautifulSoup(source, 'html.parser')

url_list=soup.select("#content > div.view > div.layout > div.series")
check=0
for i in url_list:
    # 프리미엄 카테고리 검색
    if(i.find("h3").text.find("프리미엄")>=0):
        joara_list=i.find("div")
        joara_list=joara_list.find("a")
        joara_list=joara_list.get("href")
        joara_workID=joara_list.split('=')[1]

        # url 링크 추출
        joara_url="http://www.joara.com/premium_new/book_intro.html?book_code="+joara_workID
        get_info(joara_url, joara_workID)
        check=1
        break;

if check==0:
    print("검색 결과가 없습니다.")
else:
    print("\n-------------------------\n\n검색 결과 출력 완료")

작품 검색 : 투수

-------------------------

http://www.joara.com/premium_new/book_intro.html?book_code=1427653
조아라 ID no. 1427653

-------------------------

제목: 천재 투수 민덕수
장르: 판타지
저자: [창세후]
표지 url: http://cf.joara.com/literature_file/20200214_095229.jpg_thumb.png
총 연재화수:  175
최근 업데이트일: 2020.03.05.
출간일: 20/03/05
완결 여부: 완결
조회수:  3829
추천수:  18
관심도:  18
소개: 
거구의 산골 소년이 야구공을 던지자,
모든 야구계 인사들이 눈을 크게 뜨며 외쳤다.
"아니, 저런 괴물을 어디서 데려온 거야?"

산골마을에서 동네 야구를 하던 한 소년.
기억력이 좋아 라디오 중계로 야구 용어를 배우고,
혼자서 나무에 돌맹이를 던지며 시간을 때웠다.

야구부도 없는 고교에서 특별한 인연을 만나고,
열악한 환경, 부모의 반대, 동료의 텃세.
모든 것을 이겨내고 마침내 세계 무대로 이어지는
천재투수 민덕수의 우직한 야구 일념일로(一念一路)!

강속구, 뛰어난 머리, 강심장, 포커페이스.
모든 것을 갖춘 흔들리지 않는 괴물이 세계를 놀라게 한다!

-------------------------

검색 결과 출력 완료


# 리디북스 작품 리스트 추출

In [34]:
import requests
from bs4 import BeautifulSoup

# [카테고리 : 로맨스 연재]
romance_page = 0
romance_count = 0
while True:
    romance_page += 1
    url = 'https://ridibooks.com/category/books/1650?page=' + str(romance_page)
    print(url)
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 제목 위치
    title_list=soup.select("#page_category > div.book_macro_wrapper.js_book_macro_wrapper > div > div.book_metadata_wrapper > h3 > a > span")

    # 한 페이지 당 작품 최대 20개
    for num in range(20):
        try:
            title=title_list[num].text.replace("\n","")
            title = title.split() # 제목 추출
            del title[0] # [연재] 제거
            title = " ".join(title)
            print(title)
            romance_count += 1
        except:
            break
    
    print("\n-----------------------------------------------\n")
    
    # 마지막 페이지 확인
    page_list=soup.select("#page_category > div.module_paging > ul > li.btn_next")
    if not page_list:
        break
        
# [카테고리 : 판타지 연재]
fantasy_page = 0
fantasy_count = 0
while True:
    fantasy_page += 1
    url = 'https://ridibooks.com/category/books/1750?page=' + str(fantasy_page)
    print(url)
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 제목 위치
    title_list=soup.select("#page_category > div.book_macro_wrapper.js_book_macro_wrapper > div > div.book_metadata_wrapper > h3 > a > span")

    # 한 페이지 당 작품 최대 20개
    for num in range(20):
        try:
            title=title_list[num].text.replace("\n","")
            title = title.split() # 제목 추출
            del title[0] # [연재] 제거
            title = " ".join(title)
            print(title)
            fantasy_count += 1
        except:
            break
    
    print("\n-----------------------------------------------\n")
    
    # 마지막 페이지 확인
    page_list=soup.select("#page_category > div.module_paging > ul > li.btn_next")
    if not page_list:
        break
        
# [카테고리 : BL 연재]
BL_page = 0
BL_count = 0
while True:
    BL_page += 1
    url = 'https://ridibooks.com/category/books/4051?page=' + str(BL_page)
    print(url)
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 제목 위치
    title_list=soup.select("#page_category > div.book_macro_wrapper.js_book_macro_wrapper > div > div.book_metadata_wrapper > h3 > a > span")

    # 한 페이지 당 작품 최대 20개
    for num in range(20):
        try:
            title=title_list[num].text.replace("\n","")
            title = title.split() # 제목 추출
            del title[0] # [연재] 제거
            title = " ".join(title)
            print(title)
            BL_count += 1
        except:
            break
    
    print("\n-----------------------------------------------\n")
    
    # 마지막 페이지 확인
    page_list=soup.select("#page_category > div.module_paging > ul > li.btn_next")
    if not page_list:
        break
  
        
print("로맨스 장르 총 " + str(romance_count) + "편")
print("로맨스 장르 페이지 : " + str(romance_page))
print("판타지 장르 총 " + str(fantasy_count) + "편")
print("판타지 장르 페이지 : " + str(fantasy_page))
print("BL 장르 총 " + str(BL_count) + "편")
print("BL 장르 페이지 : " + str(BL_page))

https://ridibooks.com/category/books/1650?page=1
포식자의 혼약자
계모인데, 죽음을 피하는 게 너무 쉽다
테라리움 어드벤처
디어 에데르트
낯선여름
가짜 여동생을 원하면
너밖에 모르는
먼치킨들의 총애를 받고 있습니다
파반느
나를 사랑하지 않는 그대에게
악마를 교화하는 방법
라이트 앤 다크
로잔의 가시덤불
상사와 몰래 하는 짓
그림자 없는 밤
내 죽음으로 흑화하지 마세요
내게 복종하세요
악녀에게 의리가 어디 있어
내 악역 남편을 위하여
흑막 용을 키우게 되었다

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=2
배덕 황후
폐하의 밤
미친개가 몽둥이를 물어버리면
더티스캔들
파보르 (favor)
엘리트 공주님의 수석지침서
몽중연
악마가 나를 부르면
약탈혼
가짜 성녀는 퇴장을 기다린다
비비안의 사정
불온한 결속
핸디캡 이론
이제야 나를 그리워한다
폐하가 너무 귀찮게 해
이 결혼은 어차피 망하게 되어 있다
폭군과 여우
인 더 케이지(In the CAGE)
짐승 계약
상수리나무 아래

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=3
아니라도
야수의 성
S급의 히든 퀘스트
홈, 비터 홈
낙원
해피엔딩은 없다
조연은 너나 하세요
악역 대공비의 위험한 이중생활
직장내 불순교제
싫으니까 더
롤 더 본즈(Roll the Bones)
둘이 하는 나쁜 짓
불행의 기원
어느 날 공주가 되어버렸다
도원향
뱀의 혀
스르륵
까마귀는 반짝이는 것을 좋아해
페어 트레이드(fair trade)
아폴론의 심장

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=4
매리지 앤 압생트(Marriag

완월
오아시스 내 청춘
연애, 하고 있습니까
아랫집 강짱과 윗집 늑대들
살짝궁 손 잡아드립니다
보통연애 환자와 연애할 수 있을까요
보살님은 연애중
바람 이를때면
바나나형수님
밀키러브
몽환 한자락
말썽쟁이 약혼녀
떴다 그녀
달빛연인
늑대, 그녀에게 길들여지다
나의 아름다운 야수
그저, 사랑
그녀에게 올인하다
그 남자, 날씬해진 그 여자의 사정
NEO, 박씨부인전

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=30
붉은 핏빛의 유혹
독과 나비
이계궁녀
목어화
지중해의 검은 표범
타르탄기담
레벨 업 위드 유(Level up with you)
마이 러블리 캡틴
지금 죽으러 가는 길입니다
그날의 약속
사랑할 수 밖에 없는
궁중 스캔들
구속(외전합본)
프라블럼(Problem)
사랑에 길을 잃다
애정소환
강박증
남자가 되어 돌아온, 그 녀석
센트럴빌리지의 열대야
그녀의 로맨스는 달달하다

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=31
개정판 | 썸씽 (Something)
신경 좀 꺼주실래요?
정심심 Mental Clinic
이중인격 바로잡기
사랑, 불변의 법칙
누님vs형님
꽃같은 증인보호
신데렐라와 바보온달
음침한 베라와 이상한 우주
꽃이 피어날 때
이자벨라 융커
본색
매혹 당한 그녀
황제 폐하 교육은 어떻게 하나요?
캔유 렛미인 ( Can you let me in )
아침이 오지 않아도
웨어울프
야수개
너를 갖는 건, 음악이 흐른 뒤에
오직 특별한 너

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=32
규방로맨스 - 소저의 정인
개정판 | 발코니의 그 녀석
나는 너를 원해(Je Te Veux)

이도구검(異道求劍):잘써는놈
축구 천재의 재능 폭격!
마제의 신화
주인공이 자비를 숨김
회귀자의 스타트업
나 혼자 대리사냥!
정감록
권능의 마갑
나 혼자 두배!
거침없이
다크로드 블리츠 (연재)
독표무적 (연재)
각성자
로드는 로드 중
센 놈!
슬기로운 연예생활
재벌을 향해 Suit Up!
철종
먹지 못했던 여사친들
역대급 야구 천재의 탄생

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=24
BJ대마도사
무신탄생
마이스터 - Prophet
나 혼자 천재 DNA
무한 회귀의 용사
회귀로 압도한다
대물 스카우터
링커 : 유물의 주인을 찾아드립니다
천재 광대, 톱스타되다!
불장난
골프가 좋아
500년 만에 돌아온 천마
회귀한 톱스타의 힐링라이프
최강전설 아저씨
하루가 두 번
플레이어님 귀환하신다!
월드스타 슈베르트
학자인데 너무 강함
톱스타, 그 자체
나는 폭군이다

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=25
투사의 탈을 쓴 모차르트
탈모형 미드필더
오크지만 찬양해
능력자의 포스
생명력 Max!
강남 퇴마사
폭풍의 한반도
똥볼 투수의 귀환
부검의 신 : 망자의 기억
만렙 영웅님께서 귀환하신다!
주인공이 모르는 게 없음
SSS급 마트 점원
세계는 넓고 할 일은 많지만 서자였다
축구천재 귀화하다
중원이 지다
이 맛에 재벌한다.
무협지 엑스트라가 되다
재벌가 망나니입니다만?
현대 네크로맨서 길라잡이
21세기 갑부[甲富]

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=26
용돈이 계속 늘어!
모든 것을 강화하는 헌터
게임 같은 무인도 생존기
천마재혼: 회귀로 모든 것을 얻다
배틀 킹덤 언플의

내 무공 넘사벽!
잠룡전기(潛龍傳奇)
당신의 매니저
글의 신을 향하여
우주게임의 사령관
돌아온 기간트 마스터
대물남 오천국
사상 최강의 에이스
기연무적
축구를 너무 잘함
뽑기로 강해진 SSS급 헌터
광고천재의 회귀생활
사자의 귀환
개정판 | 무림 드래곤
SSS급 마왕의 귀환
헬무트
나만이 무공을 사용할 수 있다!
튜토리얼 라이프
나 빼고 다 젊은이
나 혼자 무림 플레이어

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=51
마인예속
백작가 도련님은 회귀자
연기를 너무 잘함
야구가 너무 쉬움
이세계 최면 유희
검신, 던전 출수
탑 아이돌 락커를 꿈꾸다
SSS급 촉수괴물이 지능을 얻음!
여초 삼국지
전생 라이프
이과장은 탑스타!
재벌 닷컴
변혁 1990
무적선생
삼국지 위현전 - 죽림결의
전생에 음악의 신이었던 남자
알브레히트 일대기
수양대군, 코끼리를 만나다!
백룡의 비상
공작가의 소드마스터

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=52
페로몬맨
자고나니 세상이 게임으로 바뀌었다!
농구가 보인다!
블러드 씨커
만렙in현실
조선군벌
사마지존(邪魔至尊)
악역 바스타드입니다만
레벨업만이 살길
FFF급 구단 매니지먼트
경험치 포식자
닥터 최태수
나 혼자 통조림 사러 멀리 갔다 왔다
나 혼자 10만 대군
미래의 통장으로 재벌 되다
한냥 표국
마운드의 제왕
차원 이동으로 재벌된 남자
열혈 닥터, 명의를 향해!
이것이 법이다

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=53
3루의 귀신
눈 떠 보니 슈퍼스타
섹신!
난 대물이다
덮쳐놓고 레벨업
귀환자님 뭐 하세요?
그리스의 성신 군주
침대 위의 

역대급 마법 재능
귀신잡고 폭렙업
악마의 오른발
죽었다 깨어난 가수
비천신마
엘로스의 군주
천하제일가
내 눈에 라인이 보여
내 것만 엑스라지
미소녀를 공략하자
스킬 대장장이
갓코인
등급인생
화산권마
삼국수호전
리블 온라인
이세계 골드리치
경험 많은 뉴비
쪼렙 천마
개정판 | 페널티박스의 폭군

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=79
이세계 영주 생활
야망검
국민배우 정도현
계약이 완료되었습니다
레벨 업! 하는 식당
신흥재벌
믿음직한 1승 카드
이세계 쉐프 생존기
영혼을 노래하다
내가 슈퍼맨이 되던 날
되찾은 새벽
진짜 쩐다
기록을 부수는 타자
내가 지존이다
회귀군주
천하제일 부자
포션으로 무한성장
재능의 책
무신귀환
신검화산

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=80
풍뢰도
괴물 투수 성낙기
7번째 환생
포효강호
회귀했더니 검신이었다
이계에서 무기상
시바! 혼자 딜탱 다 하잖아!
나 혼자 극한 던전
천애협로
마도 진조휘
배우, 미친 흡입력
탑 레시피가 보여!
금검풍운(金劍風雲)
스크린의 별
천 번의 환생 끝에
전생부터 다시
오버로드 : 남들은 헬 나에겐 이지
일인전승(一人全勝)
고검독보
드래곤 레이드

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=81
신 조선:개혁의 파도
환웅의 검
다시 한 번
물약이 자꾸 땡겨
만석꾼
내 몸은 무한재생
회귀자의 스킬무쌍
마검쟁천
두 세계의 재벌
최강질주
이세계 플레이어
창천
낭왕무오
무신학사
센터 포워드
레전드 오브 노가다
라 만차의 전사
전직 폭군의 결자해지
와룡귀환
마인드 마스터

--------------------------

...

[B-가든] 소년은 내일을 죽인다
[인디고] 이렇게나 커졌어요
뽀글 사랑! ~밋밋남의 삼각관계~
달보드레
금지된 왈츠
그 사람을 조심하세요
또 하나의 세계
얼마면 돼?
[크림툰] 교활한 당신이 바꾸는 세계
도화뎐
[허니B] 비너스의 입맞춤
[허니B] 연애 콤플렉스
[허니B] 나만을 위한 니노미야
[허니B] 꼬리가 증거
[허니B] 펀치 드렁크
외로움이 부는 계절
러브&헤이트
피터팬의 거짓말
하시옌
[비애] 돈이 없어 EX

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=68
[페어리] 이런 짓 하고도 친구로 지낼 수 있을까?
[블랙스완] 듣자하니 반한 상대는 어둠 속성
[BL비엘] 섹스를 할 거라면
용사님, 에로마왕에게 사로잡히다
[페어리] 잘 될 리 없잖아
[허니B] 당신을 더럽혀도 될까요
[블랙스완] 나의 귀여운 주정뱅이
[페어리] 사랑한다고 말하면 XX다
[허니B] 일그러진 사랑의 새벽을 기다린다
[로제] 길 잃은 고양이와 사랑의 발자국
[페어리] 호스트 형무소 3호동-1000명과 하면 가석방-
[허니B] SM클럽의 소악마
[페어리] 나루세와 한 번만 할 수 있다면 죽어도 좋아
저한테 도망친다면 벌을 주겠습니다
[페어리] 좋아하는걸!
[BL비엘] 늑대씨의 발정이유
[페어리] 특이 체질 남자
엉덩이가 근질거려
[블랙스완] Trade-트레이드-
[페어리] 편도, 영화 한 편 분

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=69
[인디고] 은근 밝히는 선배와 귀하게 큰 슈가
[로제] 복숭앗빛 비트
[인디고] 짐승처럼 껴안아주마
[블랙스완] 복수는 나의 것~밤을 능욕하는 은색의 증오~
발칙한 잔업
[허니B] 12월의 앤티크 양장점
에로틱 선생님의 특별 과외수업
[페어리] 건방진 동정에게 진심일 리 없잖아
[페어리]

[해피니스B] 나사의 회전
[해피니스B] 호즈미군의 초 S한 달링
[해피니스B] 변태귀족
[해피니스B] 복숭아색 천사
[페어리] SM 과장 -변태 부하에게 개발당할 것 같아-
[페어리] 부녀자의 먹이 -내가 BL 모델이라니-
[페어리] 남자X남자=음란도수 96%
[인디고] 시종의 취향
[해피니스B] 부하가 알몸으로 갈아입으면
[BL비엘] 이모럴 데이즈
[BL비엘] 나무가 젖어드는 날의 낙원
[BL비엘] BL 성우!
[블랙스완] NAVY BLUE
오피스의 왕자님
[페어리] 라우드 왕자의 청혼
고쳐줄까? 3부
라이벌~특별한 남자
[BL비엘] 쌉쌀 바닐라
커피를 한 잔 더 부르는 노래
죽고 못사는 연애

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=86
기묘한 동거
스타와의 사랑
강을 오르는 고래
루드비히 2세
포기하는 연애
내가 네 주인이다, 이 자식아!
[B-가든] 흩날리는 벚꽃처럼
[피커스] 이 거리에서 사랑해!
별을 가진 작은 여우(리에디션)
[허니B] 남자가 되는 길
[허니B] 1인자의 욕망
마젠타
[인디고] 첫사랑 화학 실험체
[비애] 플러스 20cm의 거리
목구멍이 포도청
풍월주 - 바람과 달의 주인
[허니B] 유혹의 향기
Not so bad (낫 소 배드)
폭군 그이 -6 언더 그라운드 2-
스테이 위드 미 (Stay with me)

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=87
[페어리] 당신이라면 괜찮을 것 같습니다
없다 있으니까
열왕대전기
공심연인
[비애] 사냥감의 기분♥
그늘 핀 들꽃
러브, 라이크, 헤이트 (Love, Like, Hate)
[B-가든] 너만 좋아하고 사랑해
[BLovers] 호랑이굴 다이닝
[페어리] 꽃미남 특공대! MOYATTO
길 잃은 솜사탕
속물의 습성
[인디고] 사

망가뜨려도 괜찮아?
[페어리] 너의 몸을 연주해 즉흥적으로
[페어리] 그 녀석의 목소리
러브 호텔에서 밝혀지는 상사의 비밀
탈 것은 역시 수컷이지
[페어리] 거짓말쟁이 찰칵 찬스
[페어리] 비 온 뒤 우리는 맑음
[페어리] 다섯 개의 감각
[페어리] 월하난무
[페어리] 세 번째 키스는 진심을 담아
[페어리] BL만화는 몸으로 느끼는 거야
BL 서커스에 어서오세요
[페어리] 왕자님은 양아치를 먹어 치운다
[페어리] 너의 소리가 그치지 않아
[페어리] 여름의 잇자국
사랑과 확신의 초상
탐정이 조수를 눕힐 때까지
[페어리] 애달픈 밤의 사랑
너에게 예쁨받고 싶어
[페어리] 나에게 어울리는 굴욕을

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=106
[페어리] 채찍보단 당근으로 길들여줘
그래도 좋아하니까 잡고 싶어
[페어리] 치킨 스토커
[페어리] 말은 필요 없어
[페어리] 수컷 고양이는 그에게 안겨 운다
[페어리] 졸작 레토릭
[페어리] 옆집의 사랑스러운 스토커
[페어리] 교사가 AV 감독과 사랑해도 괜찮을까
잠들지 못하는 에도의 밤
[페어리] 독일에서 생긴 일 -Stern Stunde 찰나의 교환-
[페어리] 미남인데 유령이시네요
[페어리] 이 알의 아버지는 누구?!
애태워줘 빌리
[페어리] 묘령의 남자
[페어리] 두 사람이 마주 닿는 이야기
[페어리] 남자한테 고백할 순 없잖아
우리들의 학교 파라다이스
[페어리] 남자 며느리의 시집살이
[페어리] 거짓으로 물든 우리들의 밤
[페어리] 여름의 그림자에 작별을

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=107
[페어리] 너의 얼굴이 죽을 만큼 싫다
다리부터 안까지 조물조물하고 싶어
퓻퓻 중독 -앞뒤로 자극 합체-
너에게 먹히고 싶어
[페어리] 밤하늘의 성좌
[페어

사랑한 뚱보
남친은 우메보시 보이
괜찮나? 괜찮다, 문제 없다
합법 완구 NO.666 ~당한 만큼 되갚아라~
남동생이 매일 밤 나를 노린다
나만의 차장님~중도하차해서 만나러 와 줘~
밟아줘!!
그 녀석의 그곳을 저주합니다~사랑의 저주인형~
그 남자를 맛있게 먹는 방법 ~당신의 좋은 부분, 잘먹겠습니다~
과장님의 속옷 사정 ~내가 프릴에 흥분할 리가 없어!~
공중 화장실 남자~처음으로 하는 야외 플레이~
SNIPER
Sexy Boy~내가 게이 세계의 비너스?!~
Colors 도쿄전뇌영역
뽀뽀해줄테니까 보라구! ~저주받은 식객~
[블랙스완] 앱으로 맺어진 인연도 사랑이야
[블랙스완] 딜레마의 사랑
[블랙스완] 초보자를 위한 첫사랑 입문편
[블랙스완] 이웃집이 듣겠어!!
[블랙스완] 수습탐정 미즈키의 최악의 재앙 사건수첩

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=126
[블랙스완] 맑은 날엔 공원에서
[블랙스완] 단 것을 좋아하는 그 아이
[블랙스완] 나에게 남자친구가 생긴 특별한 이야기
[블랙스완] 건방진 왕자님은 응석쟁이
[블랙스완] 3년 전 그날부터 계속, 난 너의 장난감이었어
[블랙스완] 커플링을 사랑한 엑스트라 신데렐라
[블랙스완] 칠월의 괴담
[블랙스완] 연상 남친을 사로 잡는 방법
[블랙스완] 엇갈리는 동거생활
[블랙스완] 부드러운 온기
[블랙스완] 스트레이트★사기 테크닉
[블랙스완] 도둑고양이 회사원과 냥냥 플레이
[블랙스완] 꿈에서 깨어난 꿈
[블랙스완] 네 녀석의 그 눈은 반칙이야!!
[블랙스완] 일그러진 첫사랑
[블랙스완] 연옥의 인연
[블랙스완] 내 취향은 당신
[블랙스완] 나만을 봐줘
[블랙스완] 열혈 교생X고민쟁이 선생님
아름다운 남자 ~욕정남과 사랑받는 나~

-----------------------------------------------

https://ridibooks.com/category/book

[BL비엘] 스킵 키스 SKIP-KISS
[BL비엘] NO.1 호스트의 개인지도
[BL비엘] Colors 도쿄전뇌영역
[BL비엘] AV남자X순진남자
[BL비엘] 뽀뽀해줄테니까 보라구!
[BL비엘] 봄을 기다리는 꽃망울의 상사병
[BL비엘] 의형제에게 비밀이 있다
[BL비엘] 잔혹한 우리
[BL비엘] 너를 보고 있어
[BL비엘] HELP! 사랑의 신
[BL비엘] HELP! 사랑 바이러스
[BL비엘] 집착심
[BL비엘] 영원한 구속
[BL비엘] 서툰 네 고백을 들어줄게
[BL비엘] 그의 애인
[BL비엘] 좋아하는 걸
[BL비엘] 이상한 사랑이야기
[BL비엘] 꽃보다 동정
[BL비엘] 내가 당신을 만나고 싶은 이유
[BL비엘] 키스로 반응한 건 생리현상입니다

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=146
[BL비엘] 가련한 남친
[BL비엘] 룸메이트의 음란한 연구
[BL비엘] 해도 좋아
[BL비엘] 포르노 작가의 애완 정사
[BL비엘] 저도 함께 어떠세요?
[BL비엘] 아름다워
[BL비엘] 신령님의 제물
[BL비엘] 그 사랑, 교칙 위반!
[BL비엘] 배고픈 인큐버스
[BL비엘] 그늘에 나뭇잎 사이로 비치는 햇빛, 그리고 푸르름
[BL비엘] 음란마귀와 외톨이
[BL비엘] LOVE적
[BL비엘] 나와 고양이와 첫사랑 교차
[BL비엘] 최악의 룸셰어 시작했습니다
[BL비엘] 남자 기숙사 301호실의 연애플래그
[BL비엘] 코가미 선생님과 사오토메 군
[BL비엘] 짜증나는 녀석과 멋대로 플래그가 서버린 건
[BL비엘] 러브레터 러브레터 카페
[BL비엘] 새장 속의 파랑새
[BL비엘] 에고이스틱 블루

-----------------------------------------------

https://ridibooks.com/category/books/4050?page=147
[BL비엘] 사랑난만
[BL비엘] 싸움 최고! 키스 최고!
[B

# 조아라 작품 리스트 추출

In [1]:
import requests
from bs4 import BeautifulSoup

# [카테고리 : 로맨스 - 현재 연재 중]
joara_rom_page = 0
joara_rom = 0
while True:
    joara_rom_page += 1
    url = "http://www.joara.com/premium_new/book_list.html?page_no=" + str(joara_rom_page) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=&sub_category=&sl_cate_no=fm"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_rom_page -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        title_list = []
        romance = soup.find_all('div',{'class','list_list_con best_c'})
        romance = soup.find_all('div',{'class','finish_list_view'})
        romance = soup.find_all('b')
        
        for i in romance:
            title_list.append(i.get_text())
            
        # 중복 제거
        title_list = list(set(title_list))
        
        if '연재 작품' in title_list:
            title_list.remove('연재 작품')
        if '조회:' in title_list:
            title_list.remove('조회:')
        if '추천:' in title_list:
            title_list.remove('추천:')
        if '선작:' in title_list:
            title_list.remove('선작:')
        if 'Fax' in title_list:
            title_list.remove('Fax')
        if 'Tel' in title_list:
            title_list.remove('Tel')
        
        joara_rom += (len(title_list))
        print(title_list)
        
print("\n---------------------------------------------------------------\n")

# [카테고리 : 로맨스 - 완결]
joara_rom_end = 0
while True:
    joara_rom_end += 1
    
    url = "http://www.joara.com/premium_new/finish_list.html?page_no=" + str(joara_rom_end) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=finish&sub_category=&sl_cate_no=r"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_rom_end -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        title_list = []
        romance = soup.find_all('div',{'class','list_list_con best_c'})
        romance = soup.find_all('div',{'class','finish_list_view'})
        romance = soup.find_all('b')
        
        for i in romance:
            title_list.append(i.get_text())
            
        # 중복 제거
        title_list = list(set(title_list))
        
        if '완결 작품' in title_list:
            title_list.remove('완결 작품')
        if '조회:' in title_list:
            title_list.remove('조회:')
        if '추천:' in title_list:
            title_list.remove('추천:')
        if '선작:' in title_list:
            title_list.remove('선작:')
        if 'Fax' in title_list:
            title_list.remove('Fax')
        if 'Tel' in title_list:
            title_list.remove('Tel')
        
        joara_rom += (len(title_list))
        print(title_list)
        
print("\n---------------------------------------------------------------\n")

# [카테고리 : 판타지/무협 - 현재 연재 중]
joara_fm_page = 0
joara_fm = 0
while True:
    joara_fm_page += 1
    url = "http://www.joara.com/premium_new/book_list.html?page_no=" + str(joara_fm_page) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=&sub_category=&sl_cate_no=fm"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_fm_page -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        fm_title_list = []
        fantasy = soup.find_all('div',{'class','list_list_con best_c'})
        fantasy = soup.find_all('div',{'class','finish_list_view'})
        fantasy = soup.find_all('b')
        
        for j in fantasy:
            fm_title_list.append(j.get_text())
            
        # 중복 제거
        fm_title_list = list(set(fm_title_list))
        
        if '연재 작품' in fm_title_list:
            fm_title_list.remove('연재 작품')
        if '조회:' in fm_title_list:
            fm_title_list.remove('조회:')
        if '추천:' in fm_title_list:
            fm_title_list.remove('추천:')
        if '선작:' in fm_title_list:
            fm_title_list.remove('선작:')
        if 'Fax' in fm_title_list:
            fm_title_list.remove('Fax')
        if 'Tel' in fm_title_list:
            fm_title_list.remove('Tel')
        
        joara_fm += (len(fm_title_list))
        print(fm_title_list)

print("\n---------------------------------------------------------------\n")

# [카테고리 : 판타지/무협 - 완결]
joara_fm_end = 0
while True:
    joara_fm_end += 1
    url = "http://www.joara.com/premium_new/finish_list.html?page_no=" + str(joara_fm_end) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=finish&sub_category=&sl_cate_no=fm"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_fm_end -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        fm_title_list = []
        fantasy = soup.find_all('div',{'class','list_list_con best_c'})
        fantasy = soup.find_all('div',{'class','finish_list_view'})
        fantasy = soup.find_all('b')
        
        for j in fantasy:
            fm_title_list.append(j.get_text())
            
        # 중복 제거
        fm_title_list = list(set(fm_title_list))
        
        if '완결 작품' in fm_title_list:
            fm_title_list.remove('완결 작품')
        if '조회:' in fm_title_list:
            fm_title_list.remove('조회:')
        if '추천:' in fm_title_list:
            fm_title_list.remove('추천:')
        if '선작:' in fm_title_list:
            fm_title_list.remove('선작:')
        if 'Fax' in fm_title_list:
            fm_title_list.remove('Fax')
        if 'Tel' in fm_title_list:
            fm_title_list.remove('Tel')
        
        joara_fm += (len(fm_title_list))
        print(fm_title_list)

['알고 보니 천재 뮤지션', '멸망한 세계의 농부', '막장드라마의 제왕', '나만의 소원퀘스트 ', '블랙기업조선', '엔딩메이커', '학사신공', '환생한 도련님은 신문구독중', '평행세계의 나는 축구천재', '템복사로 인생 역전!', '피아니스트의 마을', '일념영원', '전략게임 속 엑스트라', 'Project_Mar-On', '나 혼자 미래독식 서바이벌 ', ' 환룡의 전승자', '몽골에서 유럽까지', '어떤 던전의 인간몬스터[개정판]', '함장에서 제독까지', '마신의 힘을 받은 자']
['SSS급 마검사 도련님', '능력계승 수호자', '1993 회귀재벌', '귀신 보는 남자 오덕팔', '삼국지 여포전', '시스템 에러로 종족초월', '역대급 재벌 사위', '1919년 대한광복만세 [연재]', '만렙 영웅님께서 귀환하신다!', '성진변', '헌터의 차원 농장', '카오스맨', '아르카나 마법도서관', '해피해피 고문재단', '0살부터 슈퍼스타', '시체사냥꾼 [연재]', '인생 리셋 오 소위!', '내 딸이 검술 천재', '시스템 강호지존', '환생자의 제자']
['경애하는 수령동지', '각성자', '뮤리', '혈마검신', '내가 키운 S급들', '이것이 법이다', '어서 오세요 드라큘의 흥신소에', '일신상의 이유로 잠시 휴재합니다', '훈수로 메이저리거', '좌천당한 하급 장교가 살아남는 법', '검사 김서진', '70억분의 1의 이레귤러', '재각성을 하고 보니 마왕이 되었다', '공작님, 회개해주세요!', '만렙커피:던전입구점', '로그인하자마자 VIP', '전직 용사', '각성으로 차원최강', '믿고 보는 봉감독님', '나 혼자 검술상점']
['조선, 혁명의 시대', '정령 포식자', '이번엔 진짜 재벌!', '기적의 민간요법 치료사', '아론의 귀환', '예능의 제왕', '천재의 음악방송', '회귀했더니 입대 전날', '주인공이 불행을 다 막음', '이과생 둘이 회귀했다', '걸그룹 키우는 현질러', '조자룡 하북제패', '천마 객잔'

['지천명아비무쌍', '게임 캐릭터로 이계에 떨어졌다', '포춘 콜렉터', '삼국지(三國志):더 비기닝', '어플-미래 미리보기', '봉황의 비상 - 조일전쟁편', '더 데이 : 태초의 전쟁', '삼인조(三人組)', '지옥에서 온 좌완 파이어볼러', '강호기행록 [개정증보판]', '상태창 소드마스터', '진실게임 : 배신의 덫', '민간군사기업  BLACK ANGEL', '종천지애', '심장 강탈자', '검왕까지 레벨업!', '피가 부르는 바람', '야장, 기적을 만들다', '역대급 먼치킨 마법사의 회귀', '사천당문']
['임베디드', '편의점의 소드마스터', '더블 에이스', '선혈의 군주', '미스터 매직ENT', '대제국시대', '프리랜서 마법사', '영웅의 발자취 - 두 개의 검', '싱크[Sync]', '시작하면서부터 끝나는 그와 그녀들의 학교..', '깃발 아래서', '고검독보', '검황이계정벌하다', '랍타: 이계 변호사', '용제, 신을 찢어라', '뽑기의 신이 강림하였습니다', '히든 메이지', '마장 골핑머신', '히든 직업으로 대박내기', '천년 백작: 브라운가의 비밀']
['지존', '재벌로이드 (Chaeboloid)', '21세기 무림세가', '하얀 그림자―영웅연대기', '대마법사 락스타 (Magical Rockstar)', '바람의 아들', '전능의 지배자', '얼라이브', '그레이트 써전', '신대한제국', '물리소년', '유령의 주인-유령도서관-', '월드 엔터테인먼트', '히어로는 없다', '검황도제', '맥스씨', '꿈을 찾아 왔단다', '말세 종결자', '백수연대기', '제작해서 갑질한다']
['영혼을 지배하는 플레이어', '이계십자로', '브리더(Breeder)', '노룡전기', '잠룡물용', '나는 7급 공무원이다!', '제국의 새벽', '강철의 정령술사', '영주 레이샤드', '원더풀 라이프', '환생재벌', '귀환전생', '헬 오브 더 시티 워', '끝내주는 조상 탓', '은퇴 마왕 전기 ', '판도라 이팩트', '회

['꿈 그리고 여자', '잔혹하게 농락하다', '오늘 뭐 먹지?', '나의 남편으로 명하노라', '여름 매미 우는 소리', '아이슬리드', '또 한 명의 프린세스', '안녕, 내 똥강아지', '격변의 시대', '아무것도 모르면서', '야누스의 달', '구미호에 남자가 없을리가', '나의 이름은', '왜 네게서 벗어날 수 없는가', '나의 작가님', '고양이는 아홉 번을 산다', '프러포즈', '[BL]거인의 오두막', '첫사랑과 열애 중', '주인공을 죽였습니다']
['[동양] 황제의 작은 새 (Clean Ver) 完', '나의 하늘신', '아임 소리, 레드', '봄을 만들고 있습니다', '공녀님 우리 공녀님', '너에게 흐르다', '불편한 사랑', '좋아하는데 이유가 필요해?', '사막의 나란토야', '[개정판] 미인도(美人圖) : 신수의 여인', '카모마일 한 모금', '난폭한 황제 길들이기 [완결]', '규방로맨스', '나는 마왕이로소이다', '그대의 계절', '당신은 나의 수면제', 'Mission! 구남친을 피하라', '귀묘한 카페', '죽여주는 남자', '프라이빗 하우스(Private House) 3부']
['Lost Memory', '프라이빗 하우스(Private House) 2부', '우리가 사랑하기 전(개정판)(외전 포함)', '그럼에도 우리는', '네가, 웃기를', '늑대와 여우 사이', '닥치고 결혼', '그를 유혹해(프렌즈 시리즈 2)', '나이는 숫자에 불과하다', '신시기묘사', '독과 나비', '리온, 이리 오너라!', '사랑하는 사람들', '프라이빗 하우스(Private House) 1부', '만화 같은 이야기(프렌즈 시리즈 1)', '신과 결혼했다', '사돈지간', '핫 폴리스', '거짓은 거짓으로', '밤에 오는 남자']
['당신에게필요한것을드립니다', '너에게 아포가토', '너를 찾아서 [완결]', '조련 수치가 올라갔습니다', '황제의 전직 친구입니다 [완결]', '[BL]베리테스 연애담', '용설지의 해아', '홍대 앞 로

['만 지 작', '단테의 낙원(Dante＇s Paradise)', '눈보다 아름다운*', '샤를과 베일 속의 주인', '스타의 연인', '인디언 썸머, 우리들의 행복한 계절', '사랑 정비 중', '설란', '나의 검은 공주님', '신데렐라 로맨스', '얼음성의 주인', '남자의 미학', '손끝에서 마법이', '욕정과 욕망의 사이', 'Secret Lover', '쿠키 드실래요?', '공작의 청혼', '어느 멋진 날', '아파, 당신이라서……', '톱스타, 별을 만나다']
['사랑니', '애이불비', '내 얼굴을 가진 당신', '황태자님 분부대로 (외전증보판)', '인체개조공장', '해가 뜨는 시간', '리스타트(Restart)', '미묘한 사이', '블랙 ＆ 화이트', '여기사와 용', '옷장을 열면 그곳에 바다가 ', '영원의 연인', '르웬의 세계(외전 포함)', '악마와 유리구두 (개정판)', '잔드바르의 여름', '파트너', '사랑한다, 살아간다', '책가방을 멘 신녀', '절벽 아래, 매鷹', '태권브이와 시바견']
['카르페디엠', '블랙리스트', '보나페티', '비밀소녀의 연애사', '나는 한그루입니다 ', '나는 너를 거부한다', '귀신같은 그녀', '그대의 라임라이트', '미치도록 안고싶은 너', '워너비 콤플렉스', '황후의 남자', '반짝반짝 (외전증보판)', '살아남아라, 황후! ', '마녀의 귀환 ', '바라연', '달콤한 정복', '적도의 밤', '강철의 꽃', '라스트 레볼루션', '꽃에 미치다']
['적대적 우애관계', '엄마가 되어 주세요', '봄이라 쓰고 곰이라 읽는다', '삼가 모십니다', '이 길 건너 당신에게 ', '사랑과 우정사이', '붉은 실', '끈적끈적 다가오는', '아내가 돌아왔다', '전 남편 유혹하기', '깡패오빠와 연애하기', '내일은 연애왕', '가면의 미소', '찬스', '첫사랑 우선순위', '봄날은 간절히', '아내는 베이비시터', '요아전', '사랑을 가르쳐 드립니다', '사랑과 미련의 경계선']


['SSS급 마검사 도련님', '능력계승 수호자', '1993 회귀재벌', '귀신 보는 남자 오덕팔', '삼국지 여포전', '시스템 에러로 종족초월', '역대급 재벌 사위', '1919년 대한광복만세 [연재]', '만렙 영웅님께서 귀환하신다!', '성진변', '헌터의 차원 농장', '카오스맨', '아르카나 마법도서관', '해피해피 고문재단', '0살부터 슈퍼스타', '시체사냥꾼 [연재]', '인생 리셋 오 소위!', '내 딸이 검술 천재', '시스템 강호지존', '환생자의 제자']
['경애하는 수령동지', '각성자', '뮤리', '혈마검신', '내가 키운 S급들', '이것이 법이다', '어서 오세요 드라큘의 흥신소에', '일신상의 이유로 잠시 휴재합니다', '훈수로 메이저리거', '좌천당한 하급 장교가 살아남는 법', '검사 김서진', '70억분의 1의 이레귤러', '재각성을 하고 보니 마왕이 되었다', '공작님, 회개해주세요!', '만렙커피:던전입구점', '로그인하자마자 VIP', '전직 용사', '각성으로 차원최강', '믿고 보는 봉감독님', '나 혼자 검술상점']
['조선, 혁명의 시대', '정령 포식자', '이번엔 진짜 재벌!', '기적의 민간요법 치료사', '아론의 귀환', '예능의 제왕', '천재의 음악방송', '회귀했더니 입대 전날', '주인공이 불행을 다 막음', '이과생 둘이 회귀했다', '걸그룹 키우는 현질러', '조자룡 하북제패', '천마 객잔', '튜토리얼 라이프', '회귀한 톱스타의 힐링라이프', '농사로 이계 평정', '축복의 블루스톤', '재능이 자꾸 늘어', '나 혼자 균열에 산다', '3회차 톱매니저']
['무제 관우전: 천하를 품은 구름', '기프티드 (GIFTED)', '내일은 일인자', '벌써 벗은 영주님 : 델로아 전기', '알고보니 일대일 천재', '내 안에 마교있다', '회귀자의 로열라이프', '국적없는 용병', '아기부터 시작하는 연예계 생활', '고려 상왕', '전직지존', '북경의 신룡', '회귀자의 스타트업', 

...

['씨어', '윈드 엠페러', '마스터 오브 웨폰', '미연시인데 연애를 할 수 없는 건에 대하여', '가상무공탄류', '데버스테이션 강진후', '루머 읽는 배우님', '가면의 마존', '메카니스트', '작은 아버지는 동탁 [삼국지 동황전]', '미래 나노봇을 먹었다', '히든클래스', '주먹은 강기를 싣고', '죽은 자들의 징벌', '프리매지션', '플라잉버스터', '지구를 재활용하다', '다크 프리스트', '오직 나만이 마스터다', '패도무혼']
['권능의 마갑', '나 혼자 천재 DNA', '대마법사 회장님', '시작부터 EX등급', '전설이 돌아오다', '귀환하면 세계 최강', '청소부의 행운 스텟이 폭주한다', '전신환생', '상점창 스트라이커', '제국 창건기', '무종도담', '북벌군주 효종 환생', '절대검황', '재력으로 후려치는 환생 경찰', '골프가 좋아[완결]', '붉은 제국의 축구 황제', '제온의 후예', '암묵의 지배자들(원제 : 불의 고리) [완결]', '잠자는 용', '이도구검(異道求劍):잘써는놈']
['강철 마법사', '강민혁, 세계를 먹다', '검선지가', '회귀한 헌터가 너무 강함', '회귀자의 기연 싹쓸이', '징계:응징자', '태을검제', '헨드릭의 시스템', '카드빨로 축구 스타', '역전무사', '주방의 마법사', '인피니티 클레임', '부검 스페셜리스트', '전기의 기사', '미스터 마스터 고산봉', '절대자의 푸드트럭', '복수를 위해 돌아왔다', '삼구삼진', '레벨업 건달 강대풍', '해왕전기']
['저세상 헌터의 이세상 교육', '원더풀! 마이 라이프!', '운수대통 낭인아저씨', '엘칸 스콜라', '0.0001초 광속의 이레귤러', '월드 오브 라그나로크', '이세계에서 요리사를 꿈꾼다', '검신, 던전 출수', '요동왕', '어느 기갑장교의 이야기', '영주, 황제가 되다', '꿈의 전설', '최강 귀환자의 행복 라이프', '은영', '마녀 사용설명서', '오펜스 챌린지', '죽어도 죽지 않는 남자', '

['곤왕기', '승부의 신', '길 가다 주운 SSS급 반지', '마계 이야기', '사상 최강의 마법사', '축구는 상태창으로 하는 거야', '은둔형 마법사', '무패의 랭커', '사신전설', '배우, 조선 왕이 되다', '내 디딤발이 360°', '상태창이 사라졌다.', '3루의 귀신', '제왕의 길', '내 상태창 2개', '쾌감 폭발 레벨업', '잠룡진천', '라스트 콘스탄틴', '공작가의 마마보이', '100년 수련한 축구선수']
['헌터, 웃기지마! [완결]', '마법도 이력서에 써도 되나요?', '고고학자인데 땅을 안 팜', '최강의 포수', '더 세틀러', '아이템 매니아', '주식 재벌 박태준', '난검무림', '자명공자', '블러드 씨커', '귀환전설', '어쩌다보니 이세계에 여우로 전생했습니다.', '무림 신세계', '나는 활쟁이다', '독표무적', '만렙인턴 이현성', '업어 키운 사매들', '1560 대한민국', '강남점쟁이', '국민아이돌 - 프로듀스99']
['맛깔나게 만들어드려요!', '두 세계의 재벌', '닥터 최태수', '탁구왕 김제빵', '무림전생(武林轉生)', 'S급 병사 반세주', '퍼플 미스트', '드래곤이 되다', '역대급 먼치킨 소환사의 귀환', '메이즈 헌터', '백작가 도련님은 회귀자', '인생럭키! 신유의 만화경', '나 혼자 역대급 수련', '너를 죽여야 내가 산다', '주인공 갑질만세!', '달의 미궁', '진화학개론', '포탑으로 자동사냥', '리그 오브 발할라', '풋볼 매니저 : 스탯이 보여']
['나 혼자 귀신이 보인다', '로마 : 무신의 기억', '회귀해서 레벨업', '마신 강림(魔神 降臨)', '트리니티 레볼루션', '하차 이후의 소설 속', '무한질주 - 음속의 마법사', '이번에는 정말 톱스타', '아이 엠 어 프로듀서', '스틸러 벤', '초인의 게임', '사진작가 길승우', '무공을 배우다', '딥블루 프론티어', '청염의 마왕', '술 마시다 각성한 SSS급 던전 마스터', '연기를 너무 

['주먹의 품격', '천마 하고 싶은 거 다 해!', '신들린 뮤지션', '군주, 다시 귀환하다', '삼국지 마휴전', '귀환무사', '기적의 환생', '묵객', '전공 삼국지', '시스템 스트라이커', '픽 미 업!', '마객 신마전설', '창천', '마왕의 표국', '셰도우 로드', '용문쟁천', '만학검전', '인피니티메이지', '맞을수록 강해져', '대한제국사 - 한러 전쟁']
['가문의 주인', '파워피처 만득씨', '앨버트로스', '업어 키운 걸그룹', '미칠듯한 레전드의 존엄', 'D등급 개미, SSS등급 되다!', '괴물 마법사', '포텐 폭발, 김작가!', '에이스 카드(ACE CARD)', '생존가로 각성하다!', '현대갑질', '데릴사위, 재벌 회장 되다.', '할리우드 탑스타 ', '멸망한 세계의 유일한 마법사', '그림자가 미래를 알려준다', '멸망한 세계의 사냥꾼', '철혈의 회귀자', '소환수 레벨 업!', '포제션', '스폰']
['소림곤왕', '상남자스타일', '내가 조선의 주인이다', '재벌 매니지먼트', '그랜져', '그라니트 : 용들의 땅', '퍼펙트 가이', '화산검종', '삼국지(三國志) [마행처우역거馬行處牛亦去]', '천상의 목소리', '수색 조선', '[개정판] 신룡무', '풋볼 매니지먼트', '드라마의 제왕', '낭만용병', '바보가 만드는 멋진 세상 (개정판)', '개방 방주 창업신공', '무당괴협전', '교사는 회귀해도 개고생이네[완결]', '태극검해']
['힐링송 메이커', '두 번 사는 포수', '더 캡틴 (개정판)', '탑 변호사 고판석', '만검조종', '날아라 방승준', '연금술사의 항해일지', '나는 대한민국 대통령이다', '레드썬 레볼루션', '전설이 되는 법', '역대급 재벌', '회귀의 정석', '뇌전의 군주', '다차원 파견회사', 'irns : 더 퍼스트 스텝', '법이 있다', '아빠는 탑스타!', '남자의 길', '더 스토리', '헌터 킹(개정판)']
['퍼스트 터치', '그림자 전설', 

['만렙튜터 회귀했다', '김 대리가 이렇게 일을 잘했다고?', '무한 전생(부제-스페이스 니트)', '더 디그 : 유적탐색자', '인생역전! 공돌이', '태양의 전설 바람의 노래 2부', '악마의 음악 : Other Voices', '회귀한 프로게이머', '역천의 왕', '더 라이브', '개발자의 게임', '고결한 영혼(Noble Soul)', '디멘션 -게임의 왕', '채널 마스터', '언터처블-(개정판)', '울브즈', '임해군', '포식마스터', '검귀의선', '프로듀스 좀비군단']
['데블 & 아이스', '고교파워클럽', '펜타월드', '어느 대마도사의 회귀록', '이질자', '미친 재능의 마도사', '더 러너 : 마법포식자', '천마사냥꾼', '패전처리, 회귀하다', '신석기 마스터', '공간지배자 - 이계를 삼키다', '일인전승(一人全勝)', '서쳐 - 볼수록 능력치 올라', '마수 지배자', '용무쌍', '혼혈황제의 회귀록', '마법상점의 대마법사', '재벌 닷컴', '그땐 득템인 줄 몰랐다', '나만 공략법을 알고있다']
['H.I.D', '꿈꾸는 작사가', '절세신검존', '용병단장 회귀하다', '내나이 90살에 회귀라니!', '천마님 부활하셨도다', '바운더리', '돌아온 쩔컨 트리오', '불사무적', '21세기 마샬아츠', 'Hey! 철가방', '소공자로 환생한 그랜드마스터', '샐러리맨과 킬러', '먼치킨의 귀환', '자동으로 수련되는 남자', '폭주상인', '법정에서 봅시다', '시스템 창 지배자', '21세기 현대 마법전서', '금검풍운(金劍風雲)']
['일천회귀록', '죽음의 봄', '어게인 마이라이프', '아크(ARK)', '트릭아이', '쥬빌로기', '불사자 : 회귀해서 죽는다', '9레벨이지만 괜찮아', '선달환생', '역대급 집사', '회귀했더니 던전', '전투왕', '요시카와 에이지의 삼국지', '탑 레시피가 보여!', '황금백수', '윈드 오브 파이어', '쟁자수', '공동전인', '인트로 코드', '랜더 (완결)']
[

['적룡마제', '우뢰검', '역검[易劍]', '이계용사 귀환기', '제라스의 여관', '강남무원', '소림삼십칠방', '던전 드림', '크로스카운터', '아수라', '학사 김필도', '더블커넥션', '테이머 루트', '듀얼 로드', '직장은 판타지다.', '사샤와 레인', '퍼펙트런', '운룡쟁천', 'IT제왕 한강수', '영광의 포뮬러']
['던전 카드사용백서', '블러드 킹', '변신술사 카이', '천뢰무한', '드래곤 남매', '사상 최강의 영웅(개정판)', '듀얼라이프', '이계정벌기', '무림감찰단', '부활 쓰는 성기사', '무적지존', '데모닉', '아켄', '마중마제', '엘란', '마협 소운강', '대법왕', '오메가', '던전 십팔기', '몬스터로드']
['철자검', '자베스', '천마봉', '낭인무적', '연기의 신', '죽지않는 자(현대 판타지)', '만석꾼', '궁극의 쉐프', '던전 키퍼(Dungeon Keeper)', '신존', '리한', '성모전사', '멋진 인생', '광풍무', '블러드로드', '팔부전사', '태평천하', '무적군림보', '고스트아머', 'TOP']
['오리진 1st', '지배하는 자', '소명전기', '비불전사', '낭왕', '내 전투력 10000000000', '신마강림', '브레이크 마스터', '지존육성 프로젝트', '태극검선', '도사 청수', '내 안에 몬스터 있다.', '풍운만리 만학전', '초월의 플레이어', '로드 오브 마스터', '질주장가', '단천붕지', '천검', '베스트원', '귀향검']
['바바리안', '내 현실은 판타지', '열혈마스터', '메이저리거', '내가 법이다', '건달의제국', '신성괴의', '게이트 러너', '혈도제', '천하공부출소림', '광신광세', '그라운드의 사령관', '서바이벌', '하이로드', '프레어백작', '무한각성', '<2016>시크릿 플레이어 (完)', '풍주', '일로정진', '악독한']
['왕의투구', '광존', '효우', '극양극혈', '귀환', '불

In [2]:
print(joara_rom_page)
print(joara_rom_end)
print(joara_rom)
print(joara_fm_page)
print(joara_fm_end)
print(joara_fm)

37
87
2447
37
189
4489
